In [1]:
from fastai.vision.all import *
import fiftyone as fo

In [5]:
BASE_TRAIN_IMAGES_PATH = Path("../data/train_images/")
BASE_TEST_IMAGES_PATH = Path("../data/test_images/")
DATASET_NAME = "paddy-dataset"
TRAIN_DF_PATH = "../data/train.csv"
EXISTS = True if DATASET_NAME in fo.list_datasets() else False

In [6]:
# Create an empty dataset
dataset = fo.Dataset(DATASET_NAME, _create = not EXISTS)
print(f"Created a dataset\n{dataset}")

# Read the annotatinos file
train_df = pd.read_csv(TRAIN_DF_PATH)
train_df.head(2)

Created a dataset
Name:        paddy-dataset
Media type:  None
Num samples: 0
Persistent:  False
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)


,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45


In [8]:
# If the dataset does not exist already, then populate the Dataset with train and test images
if not EXISTS:
    for row in progress_bar(list(train_df.itertuples())):
        
        # Create a sample
        sample = fo.Sample(filepath = BASE_TRAIN_IMAGES_PATH/row[2]/row[1])
        
        # Assign the fields to our sample
        sample["variety"] = row[3]
        sample["age"] = row[4]
        sample["dset_type"] = "train"
        
        # Assign the ground truth label to our sample
        sample["ground_truth"] = fo.Classification(label = row[2])
        
        # Add this sample to our dataset
        dataset.add_sample(sample)
        
    
    test_images = [x for x in BASE_TEST_IMAGES_PATH.glob("**/*") if x.is_file()]
    for img in progress_bar(test_images):
        
        # Create a sample
        sample = fo.Sample(filepath = img)
        
        # Assign the dset type for this sample
        sample["dset_type"] = "test"
        
        # Add the sample to our dataset
        dataset.add_sample(sample)
        

    # Compute the metadata on the dataset
    dataset.compute_metadata(overwrite = True)

Computing image metadata...
 100% |█████████████| 13876/13876 [21.0s elapsed, 0s remaining, 726.4 samples/s]      


In [10]:
session = fo.launch_app(dataset, port = 8085)